In [ ]:
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA
from sklearn import feature_selection
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


def rf_classifier(X, y, random_state=0):
    from sklearn.ensemble import RandomForestClassifier
    
    np.random.seed(random_state)
    arr = np.arange(0,np.shape(y)[0],1)
    np.random.shuffle(arr)
    
    y = y[arr]
    X = X[arr,:]


    model = RandomForestClassifier()
    model.fit(X, y)
        
    return model
    
def svm_classifier(X, y, random_state=0):
    from sklearn.svm import SVC
    
    np.random.seed(random_state)
    arr = np.arange(0,np.shape(y)[0],1)
    np.random.shuffle(arr)
    
    y = y[arr]
    X = X[arr,:]


    model = SVC(kernel = "rbf",random_state=random_state)
    model.fit(X, y)

    return model
    
    
def lda_classifier(X, y, random_state=0):
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    
    np.random.seed(random_state)
    arr = np.arange(0,np.shape(y)[0],1)
    np.random.shuffle(arr)
    
    y = y[arr]
    X = X[arr,:]

    model = LinearDiscriminantAnalysis()
    model.fit(X, y)
    
    return model

def xgboost_classifier(X, y, random_state=0):
    import xgboost
    
    np.random.seed(random_state)
    arr = np.arange(0,np.shape(y)[0],1)
    np.random.shuffle(arr)
    
    y = y[arr]
    X = X[arr,:]

    model = xgboost.XGBClassifier(n_jobs=8)
    model.fit(X, y)
    
    return model

def normalize(msi):
    
    normalised_msi = msi.copy()
    
    for i in range(0,np.shape(normalised_msi)[1]):
        normalised_msi[:,i] = msi[:,i]/np.sum(msi[:,i])
        
    normalised_msi[np.isnan(normalised_msi)] = 0
        
    return normalised_msi

In [ ]:

mass_range = [200,1400]
tol = 0.1
nb_classes = 2

balanced_accuracy_vec = []
for random_state in range(1,16):
    
    df = pd.read_csv("/media/USB2/DL_MASS/Dataset/DS/Annot_table_all.csv",sep=",", header=0)
    df["train"] = 1
    f = "/media/USB2/DL_MASS/Dataset/DS/MSI/centroid_data/param_3/mat_bin_vec.npy"
    mat_bin_vec = np.load(f)
    model = train_cnn(mat_bin_vec,df,nb_classes,tol,random_state)
    df = pd.read_csv("/media/USB2/DL_MASS/Dataset/DS/pseudomnas_control_annottable.csv",sep=",", header=0)
    df = df.loc[df["train"]==1].copy()
    df["train"] = 0
    f = "/media/USB2/DL_MASS/Dataset/DS/MSI/centroid_data/param_microcol/mat_bin_vec.npy"
    mat_bin_vec = np.load(f)
    y_proba,y_classes,balanced_accuracy = evaluate(model,df,mat_bin_vec,nb_classes)
    balanced_accuracy_vec.append(balanced_accuracy)
    
results_pred_df = pd.DataFrame()

vec_im_names = []
for i in range(1,16):
    im_name = "eval2micro" + str(i)
    vec_im_names.append(im_name)

results_pred_df["Analysis file name"] = vec_im_names
results_pred_df["Analysis"] = "CNN train on all"
results_pred_df["Iteration"] = range(1,16)
results_pred_df["Balanced accuracy"] = balanced_accuracy_vec

results_pred_df.to_csv('/media/USB2/DL_MASS/Dataset/DS/results/CNN_trainall_eval2micro_pred_df.csv',index=False)